In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from natsort import natsorted
import numpy as np
import matplotlib.animation as animation
import xarray as xr
import imageio
import matplotlib
import gc
import sys
from matplotlib.colors import LinearSegmentedColormap

import matplotlib.colors as mcolors

# Assuming /AutomationModule is in the root directory of your project
sys.path.append(os.path.abspath(r'H:\phd stuff\tidy3d'))

from AutomationModule import * 

import AutomationModule as AM

tidy3dAPI = os.environ["API_TIDY3D_KEY"]

In [2]:
def create_movie(field_time_out, monitor_lambdas,name='',type='t',log=False,path="",frames_per_second=1,rem_frames=False, normalize=True, a=5/3):
    frames = []

    max_values = field_time_out.max(axis=(0,1,2)) if normalize else 1

    field_time_out = field_time_out/max_values
    field_time_out['x'] = field_time_out['x']/a
    field_time_out['y'] = field_time_out['y']/a
    field_time_out['z'] = field_time_out['z']/a


    if log:
        field_log = np.log10((field_time_out))
        folder_pics = "logPics"
        
    else:
        field_log = ((field_time_out))
        folder_pics = "linPics"

    if not os.path.exists(f'{path}/{folder_pics}'):
            os.makedirs(f'{path}/{folder_pics}')
            print(f"Folder {path}/{folder_pics} created successfully.")




    for i in range(np.shape(field_time_out)[-1]):
        if os.path.isfile(f'{path}/{folder_pics}/frame_{i}.png'):
            frames.append(f'{path}/{folder_pics}/frame_{i}.png')
            continue
        try:
            fig, ax = plt.subplots(figsize=(14/2, 18/2))
            #fig, ax = plt.subplots(figsize=(12, 6))
            if type=="t":
                pcolormesh = (field_log).isel(t=i).squeeze().plot.pcolormesh(ax=ax,cmap="jet")
            else:
                pcolormesh = (field_log).isel(f=i).squeeze().plot.pcolormesh(ax=ax,cmap="jet")

            ax.set_aspect('auto', adjustable='box')
            #plt.ylim(-50,50)
            try:
                plt.title(f'Time: {str(np.array(field_time_out['t'][()][i])*1e12)} ps')
            except:
                plt.title(f'$\\nu$: {(1/np.array(td.C_0/field_time_out['f'][()][i])):.4g}')


            # Save the frame
            plt.savefig(f'{path}/{folder_pics}/frame_{i}.png')
            plt.close(fig)
            frames.append(f'{path}/{folder_pics}/frame_{i}.png')
        except:
            break
        

    name_movie = f'{path}/{name}.mp4' if name else f'output/anderson/d(t) analysis/Diameter d(t) at output of the structure Range - {monitor_lambdas[0]:.3g} - {monitor_lambdas[-1]:.3g}.mp4'
    with imageio.get_writer(name_movie, fps=frames_per_second) as writer:
        for frame in frames:
            image = imageio.imread(frame)
            writer.append_data(image)

    # Optionally, remove the individual frames if no longer needed
    if rem_frames:
        for frame in frames:
            os.remove(frame)
    
    return False

In [3]:
import scipy.integrate
def IPR(intensity_array,x,y):
    intensity = intensity_array
    integral_1 = scipy.integrate.trapezoid(scipy.integrate.trapezoid(intensity**2, x=y, axis=1), x=x, axis=0)
    integral_2 = scipy.integrate.trapezoid(scipy.integrate.trapezoid(intensity, x=y, axis=1), x=x, axis=0)**2

    return ((integral_1/integral_2))

def calculate_IPR(values,x1):
    integral_1 = scipy.integrate.simpson(values**2, x=x1, axis=0)
    integral_2 = scipy.integrate.simpson(values, x=x1,axis=0)**2

    return integral_1/integral_2

In [4]:
a=1
chi = [0.2,0.3,0.31,0.32,0.33,0.34,0.35,0.36,0.37,0.38,0.39,0.40]

In [ ]:
folders = [
        # rf"H:\phd stuff\tidy3d\data\09_11_2024 Beam Spreading 0.18 - 0.52 Larger Cell",
        # rf"H:\phd stuff\tidy3d\data\09_11_2024 Beam Spreading 0.18 - 0.52"
        # rf"H:\phd stuff\tidy3d\data\09_12_2024 Beam Spreading 0.2 - 0.48 Average"
        # rf"H:\phd stuff\tidy3d\data\09_13_2024 Beam Spreading 0.15 - 0.3 Average"
        rf"H:\phd stuff\tidy3d\data\09_13_2024 Beam Spreading 0.24 - 0.44 Average"
        # rf"H:\phd stuff\tidy3d\data\09_13_2024 Beam Spreading 0.24 - 0.44 Average Periodic Conditions"
    ]

for item in folders:
    folder_path = item
    # fig2, axs2 = plt.subplots(len(chi)+1, 1, figsize=(15, 10), sharex=True)o
    
    data = []
    for i,foldername in enumerate(natsorted(os.listdir(folder_path))):
        intensities = []
        intensities_exit = []
        xis = []
        xis_exit = []
        print(foldername)
        # plt.figure(figsize=(15, 10))
        file=os.path.join(folder_path, foldername)
        samples = []
        for j,filename in enumerate(natsorted(os.listdir(file))):
            print(filename)

            samples += [j]
            fileSim=os.path.join(file, filename)
            sim_result = (AM.loadFromFile(key = tidy3dAPI, file_path=fileSim))
            intensity = sim_result.sim_data.get_intensity("freq_monitorFieldOut")
            lambdas = 1/sim_result.monitor_lambdas



            #create_movie(intensity,a/lambdas,name=f"logfreqmonitor {lambdas[0]:.3g} - {lambdas[-1]:.3g}",type="f",log=True,path=rf"H:\phd stuff\tidy3d\output/2D SHU/Freq Test Spreading/{Path(filename).stem}",frames_per_second=1,normalize=True,a=a)



            intensity_slab =  intensity[np.where(np.bool_(intensity["x"]>=-49) & np.bool_(intensity["x"]<=49))[0],np.where(np.bool_(intensity["y"]>=-12.45) & np.bool_(intensity["y"]<=12.5))[0],:,:]*1e16
            intensities += [intensity_slab]
            intensities_exit += [intensity_slab[:,-1,:].squeeze()]
            xis +=[2*np.sqrt(IPR(intensity_slab,intensity_slab.x,intensity_slab.y).squeeze())]
            xis_exit +=[calculate_IPR(intensity_slab[:,-1,:].squeeze(),intensity_slab.x)]
        data += [{"xis_exit":xis_exit,"xis_slabs":xis,"intensities_slab":intensities,"data":np.mean(intensities,axis=0),"lambdas":lambdas, "foldername":foldername,"xis":np.mean(xis,axis=0),"chi":chi[i], "samples":samples},]
        break
    break

       



In [ ]:
fig1, axs1 = plt.subplots(2, 1, figsize=(15, 10), sharex=True, sharey=True)
# fig1, axs1 = plt.subplots(np.shape(data)[0], 1, figsize=(10, 30), sharex=True, sharey=True)
j=0
for i,item in enumerate(data):

 ax = axs1[j]
 j=j+1

 for xi_slab in item["xis_exit"]:
    ax.scatter(intensity.f/td.C_0,xi_slab)
 
 ax.plot(intensity.f/td.C_0,np.mean(np.array(item["xis_exit"]),axis=0),"--bo",label=fr"Mean $a/\xi$",color="#000435")
 
 ax.legend()

 
 ax.set_title(rf"$\chi = {item["chi"]}$", fontsize=14)
 ax.set_ylabel(rf"$a/\xi$")

plt.xlabel(rf"$\nu$")
plt.tight_layout()

plt.show()
    

In [ ]:
colors = [
          (1, 1, 1),  # White
          (0, 0, 1),  # Blue
          (1, 1, 0),  # Yellow
          (0, 1, 0),  # Green
          (1, 0, 0), #Red
          (0, 0, 0), # Black
          ] 
n_bins = 500  # Number of bins for smooth transition
cmap = mcolors.LinearSegmentedColormap.from_list("custom_colormap", colors, N=n_bins)
f = (sim_result.sim_data.get_intensity("freq_monitorFieldOut").f)/td.C_0
all_data = np.array([np.array(item["xis_exit"]) for i, item in enumerate(data) if chi[i] in [0.2, 0.3, 0.35]])
# all_data = np.array([np.array(item["xis_exit"]) for i, item in enumerate(data)])
global_vmin = np.min(all_data)
global_vmax = np.max(all_data)

fig1, axs1 = plt.subplots(2, 1, figsize=(15, 10), sharex=True, sharey=True)
# fig1, axs1 = plt.subplots(np.shape(data)[0], 1, figsize=(7, 30), sharex=True, sharey=True)
j=0
for i,item in enumerate(data):
 
 ax = axs1[j]
 j=j+1
 ax.set_title(rf"$\chi = {item["chi"]}$", fontsize=14)
 im=ax.imshow(np.array(item["xis_exit"]), vmin = global_vmin, vmax = global_vmax, 
                         extent =[np.min(f),np.max(f), np.min(item["samples"]),np.max(item["samples"])], 
                            interpolation ='gaussian', origin ='lower', cmap=cmap,aspect='auto') 
 cbar = fig1.colorbar(im, ax=ax, orientation='vertical', fraction=0.02, pad=0.04,label=rf"$a/ \xi$")
 ax.set_ylabel(rf"Sample Number")

axs1[-1].set_title(rf"$\chi = {0.4}$", fontsize=14)

plt.xlabel(rf"$\nu$")
fig1.legend(fontsize="20")
plt.tight_layout()

plt.show()

In [ ]:
print(np.shape(data))
chi = [0.3,0.31,0.32,0.33,0.34,0.35,0.36,0.37,0.38,0.39,0.40]
# chi = [0.3,0.31,0.32,0.33,0.34]


In [9]:
for item in data[1:]:
    print(item["chi"])

In [10]:
field = np.array([(item["xis"]) for item in data[1:]])

In [ ]:
np.shape(field)

In [12]:


# field_loc_length = xr.DataArray(field, coords={"f": (sim_result.sim_data.get_intensity("freq_monitorFieldOut").f)/td.C_0,"chi":np.array(chi)}, dims=["chi","f"],attrs=dict(
#         description=rf"$a/\xi$",
#     ),)


In [ ]:
f = (sim_result.sim_data.get_intensity("freq_monitorFieldOut").f)/td.C_0
fig, ax = plt.subplots(figsize=(14,9))
# Define the colors: Blue -> White -> Yellow -> Green -> Red
colors = [
          (1, 1, 1),  # White
          (0, 0, 1),  # Blue
          (1, 1, 0),  # Yellow
          (0, 1, 0),  # Green
          (1, 0, 0), #Red
          (0, 0, 0), # Black
          ]  

# Create a custom colormap
n_bins = 500  # Number of bins for smooth transition
cmap = mcolors.LinearSegmentedColormap.from_list("custom_colormap", colors, N=n_bins)

# (field_loc_length).plot.pcolormesh(ax=ax,cmap=cmap)
im=plt.imshow(field, vmin = np.min(field), vmax = np.max(field), 
                 extent =[np.min(f),np.max(f), np.min(chi),np.max(chi)], 
                    interpolation ='gaussian', origin ='lower', cmap=cmap,aspect='equal') 
# Add colorbar for reference
cbar = fig.colorbar(im, ax=ax, label=rf"$a/ \xi$",orientation='vertical', fraction=0.02, pad=0.04)

plt.xlabel(rf"$\nu$")
plt.ylabel(rf"$\chi$")

plt.show()